In [6]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
import pandas as pd
from IPython.display import display as display_dataframe, clear_output
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
import keras
# from keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
import cv2
import os
import seaborn as sns
from itertools import combinations
from pympler.tracker import SummaryTracker
from shutil import copyfile
# from keras.preprocessing.image import load_img, img_to_array


import warnings
warnings.filterwarnings('ignore')

def display_image(image, title=''):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# load data

In [2]:
#vgg can't take input of 28x28 so we shall resize all our images
pickle_folder = "pickle files"
data = pickle.load(open('data.pickle', 'rb'))
print("Data of format:")
for ch in data:
    print("data_length(" + ch + ") =", len(data[ch]))

#no. of classes
classes = len(data.keys())

Data of format:
data_length(छ) = 2000
data_length(ह) = 2000
data_length(भ) = 2000
data_length(घ) = 2000
data_length(ब) = 2000
data_length(व) = 2000
data_length(क) = 2000
data_length(ट) = 2000
data_length(ज) = 2000
data_length(ग) = 2000
data_length(ज्ञ) = 2000
data_length(ढ) = 2000
data_length(ठ) = 2000
data_length(ख) = 2000
data_length(क्ष) = 2000
data_length(य) = 2000
data_length(र) = 2000
data_length(झ) = 2000
data_length(प) = 2000
data_length(ष) = 2000
data_length(ञ) = 2000
data_length(ड) = 2000
data_length(त) = 2000
data_length(न) = 2000
data_length(ङ) = 2000
data_length(म) = 2000
data_length(द) = 2000
data_length(स) = 2000
data_length(फ) = 2000
data_length(च) = 2000
data_length(ण) = 2000
data_length(ध) = 2000
data_length(श) = 2000
data_length(थ) = 2000
data_length(त्र) = 2000
data_length(ल) = 2000


# Rearranging data in input-output format

In [3]:
file = "class_numerals_{}.pickle".format(classes)
if file in os.listdir(pickle_folder):
    class_numerals = pickle.load(open(os.path.join(pickle_folder, file), 'rb'))
    print("Unpickled")
else:
    class_numerals = {i:list(data.keys())[i] for i in range(classes)}
    pickle.dump(class_numerals, open(os.path.join(pickle_folder, file), 'wb'))
    print("Error. Incompatible. Pickled new file:", file)
def character_to_encoding(char, classes=classes):
    for i in class_numerals:
        if class_numerals[i] == char:
            return np_utils.to_categorical(i, classes)

def encoding_to_character(encoding):
    return class_numerals[encoding.argmax()]

Unpickled


# CNN

In [4]:
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("GPUs:", k.tensorflow_backend._get_available_gpus())

# print("input shape:", input_shape)
print("classes:", classes)

def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

GPUs: ['/job:localhost/replica:0/task:0/device:GPU:0']
classes: 36


# For 2 classes

In [ ]:
%%time

batch_size = 128
num_classes = classes
epochs = 500
k_r = 0.01
b_r = 0.01
dropout = 0.1

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
input_shape=(32, 32, 1)
results = pd.DataFrame(columns=['Char', 'prob'])
# # display_dataframe(results)

count = 0

# tracker = SummaryTracker()

for characters in combinations(list(data.keys()), 2):
    #selecting labels
#     labels = list(data.keys())
#     np.random.shuffle(labels)
#     labels = labels[:number_of_characters_at_once]
#     characters = ['घ', 'ध']
#     print("\n", iteration, labels)

    #preparing relevant data
    X = []
    y = []
    for ch in characters:
        X += data[ch]
        y += [ch] * len(data[ch])
    print(characters)

    #resizing input image and preprocessin
    X = np.array(list(map(lambda x: cv2.resize(x/255, (input_shape[0], input_shape[1])).reshape((input_shape[0], input_shape[1], 1)), X)))
    y_ = np.array(list(map(lambda x: character_to_encoding(x, classes), y)))
#     print(X.shape, y_.shape)

    #train test split
    train_ratio = 0.8
    X_train, X_test, y_train, y_test = train_test_split(X, y_, train_size=0.8, random_state=20, shuffle=True, stratify=y)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


    #the model itself
    model = Sequential()

    model.add(Conv2D(6, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r),
                     input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))
    model.add(Conv2D(12, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)
                    ))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))
    model.add(Conv2D(120, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)
                    ))

    # NN
    model.add(Flatten())
    #hidden layer 1
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal',
                    kernel_regularizer=keras.regularizers.l2(k_r)))
    model.add(Dropout(dropout))
    #output laye1
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    print("\n############################")
    print("## Memory usage:", get_model_memory_usage(batch_size=batch_size, model=model), "GB ##")
    print("############################\n")

    #adding custom weights
    pre_weights = keras.models.load_model(os.path.join(pickle_folder, "lenet_relu_3_conv_layer.h5")).get_weights()
    model.set_weights(pre_weights)

    #run model
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.15,
            callbacks = [early_stopping_callback])

    #saving results
    a = model.predict(X_test)
    print(characters)
    char_indices_in_encodings = [character_to_encoding(characters[i]).argmax() for i in range(2)]
    results = pd.DataFrame(a[:, char_indices_in_encodings], columns = characters)
    dummy_y_test = np.array(list(map(encoding_to_character, y_test)))
    output = np.array(dummy_y_test)
    for ch in characters:
        output[results[ch] > 0.5] = ch
    results['true'] = np.array(list(map(str, output == dummy_y_test)))
#     display_dataframe(results)
    results.to_excel('pickle files/binary_classification_excel_files/' + '_'.join(characters) +
                     '_' + str(model.evaluate(X_test, y_test)[1]) + '.xlsx')
    
    del X, y, y_, X_train, y_train, X_test, y_test, results, dummy_y_test, a, output, model, pre_weights
    k.clear_session()
#     tracker.print_diff()
#     break

In [ ]:
folder = 'results/part classifiers/2/'
file_list = []
accuracy_list = []

for file in os.listdir(folder):
    if file in ['better_ones', 'worse_ones']:
        print("skipping folder", file)
        continue
    file_list.append(file)
    accuracy_list.append(float(file[:-5].split('_')[2]))
    
from shutil import copyfile
#dataframe
file_to_accuracy_frame = pd.DataFrame({'file': file_list, 'accuracy': accuracy_list}).sort_values('accuracy')
# display_dataframe(file_to_accuracy_frame.head(10))
for file in file_to_accuracy_frame['file'].tail(10):
    src = folder + file
    dst = folder + 'better_ones/' + file
    copyfile(src, dst)

# for 3 to 35 classes

In [5]:
%%time

def train_lenet(characters):
    batch_size = 128
    num_classes = classes
    epochs = 500
    k_r = 0.01
    b_r = 0.01
    dropout = 0.1

    early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
    input_shape=(32, 32, 1)
    results = pd.DataFrame(columns=['Char', 'prob'])
    # # display_dataframe(results)

#     count = 0
    X = []
    y = []
    for ch in characters:
        X += data[ch]
        y += [ch] * len(data[ch])
    print(characters)

    #resizing input image and preprocessin
    X = np.array(list(map(lambda x: cv2.resize(x/255, (input_shape[0], input_shape[1])).reshape((input_shape[0], input_shape[1], 1)), X)))
    y_ = np.array(list(map(lambda x: character_to_encoding(x, classes), y)))
#     print(X.shape, y_.shape)

    #train test split
    train_ratio = 0.8
    X_train, X_test, y_train, y_test = train_test_split(X, y_, train_size=0.8, random_state=20, shuffle=True, stratify=y)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


    #the model itself
    model = Sequential()

    model.add(Conv2D(6, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r),
                     input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))
    model.add(Conv2D(12, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)
                    ))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))
    model.add(Conv2D(120, kernel_size=(5, 5),
                     activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)
                    ))

    # NN
    model.add(Flatten())
    #hidden layer 1
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal',
                    kernel_regularizer=keras.regularizers.l2(k_r)))
    model.add(Dropout(dropout))
    #output laye1
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    print("\n############################")
    print("## Memory usage:", get_model_memory_usage(batch_size=batch_size, model=model), "GB ##")
    print("############################\n")

    #adding custom weights
    pre_weights = keras.models.load_model(os.path.join(pickle_folder, "lenet_relu_3_conv_layer.h5")).get_weights()
    model.set_weights(pre_weights)

    #run model
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.15,
            callbacks = [early_stopping_callback])
    
    #saving results
    a = model.predict(X_test)
#     print(characters)
    char_indices_in_encodings = [character_to_encoding(characters[i]).argmax() for i in range(len(characters))]
    results = pd.DataFrame(a[:, char_indices_in_encodings], columns = characters)
    dummy_y_test = np.array(list(map(encoding_to_character, y_test)))
    output = np.array(dummy_y_test)
    for ch in characters:
        output[results[ch] > 0.5] = ch
    results['true'] = np.array(list(map(str, output == dummy_y_test)))
#     display_dataframe(results)
    return results

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 13.6 µs


In [8]:
folder = 'results/part classifiers/'
for number_of_character_classes in range(19, 36):
    #declare folder variables for easier accessibility
    current_folder = folder + str(number_of_character_classes) + '/'
    better_folder = current_folder + 'better_ones/'
    worse_folder = current_folder + 'worse_ones/'
    previous_folder = folder + str(number_of_character_classes - 1) + '/'
    previous_better_folder = previous_folder + 'better_ones/'
    previous_worse_folder = previous_folder + 'worse_ones/'
    
    #good and bad characters obtained in the previous run
    previous_better_characters = []
    previous_worse_characters = []
    
    #previous good and bad characters
    for file in os.listdir(previous_better_folder):
        previous_better_characters.append(file.split("_")[:number_of_character_classes-1])
    for file in os.listdir(previous_worse_folder):
        previous_worse_characters.append(file.split("_")[:number_of_character_classes-1])
    
    already_done = list(map(lambda x: set(x[:-5].split('_')[:-1]), os.listdir(current_folder)))
#     print(already_done)
#     break
    
    print(previous_better_characters)    
    print(previous_worse_characters) 

    #run a LeNet for every set
    count = 0
    for characters in previous_better_characters + previous_worse_characters:
        for ch in data.keys():
            if set(characters + [ch]) not in already_done:
                if ch not in characters:
                    results = train_lenet(characters + [ch])
                    results.to_excel(current_folder + '_'.join(characters+[ch]) +
                                     '_' + str(sum(results['true'] == 'True')/results.shape[0]) + '.xlsx')
                    k.clear_session()
                    count += 1
                    print(count, number_of_character_classes)
            else:
              print(characters+[ch], "already done!")
              count+=1
        clear_output()
    
    #arranging good and bad excel files.
    file_list = []
    accuracy_list = []

    for file in os.listdir(current_folder):
        if file in ['better_ones', 'worse_ones']:
            print("skipping folder", file)
            continue
        file_list.append(file)
        accuracy_list.append(float(file[:-5].split('_')[number_of_character_classes]))

    #dataframe
    file_to_accuracy_frame = pd.DataFrame({'file': file_list, 'accuracy': accuracy_list}).sort_values('accuracy')
#     for display_dataframe(file_to_accuracy_frame.head(10))
    for file in file_to_accuracy_frame['file'].head(10):
        src = current_folder + file
        dst = worse_folder + file
        copyfile(src, dst)
        
    for file in file_to_accuracy_frame['file'].tail(10):
        src = current_folder + file
        dst = better_folder + file
        copyfile(src, dst)
#     break

skipping folder better_ones
skipping folder worse_ones
[['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'ट', 'स', 'य'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'क्ष', 'ङ', 'ख', 'ग', 'छ', 'म', 'ह', 'ट'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'य', 'ह', 'ज'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'ट', 'य', 'ह'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'क्ष', 'ङ', 'ख', 'ग', 'छ', 'ठ', 'म', 'स'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'य', 'ह', 'ज्ञ'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'य', 'ह', 'म'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'य', 'ट', 'ह'], ['ञ', 'ढ', 'ष', 'झ', 'ल', 'च', 'ण', 'ध', 'र', 'श', 'फ', 'त', 'ख', 'ङ', 'ठ', 'क्ष', 'ग', 'य', 'ह', 'ट'], ['ञ', 

KeyboardInterrupt: 